In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
import os

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [2]:
def get_video_files(directory):
    video_files = []
    subfolders = ['deceptive', 'truthful']
    for subfolder in subfolders:
        path = os.path.join(directory, subfolder)
        files = [os.path.join(subfolder, f) for f in os.listdir(path) if f.endswith('_metrics.csv')]
        video_files.extend(files)
    return video_files

In [3]:
def load_data_from_files(file_list, directory):
    data_frames = []
    for file in file_list:
        path = os.path.join(directory, file)
        df = pd.read_csv(path)
        pd.get_dummies(df['Emotion']) # Must convert to one-hot encoding here to avoid issues with missing values/columns
        data_frames.append(df)
    return pd.concat(data_frames, ignore_index=True)

In [4]:
video_metric_directory = '../video_metrics/'
video_files = get_video_files(video_metric_directory)
train_files, test_files = train_test_split(video_files, test_size=0.2, random_state=42)

train_data = load_data_from_files(train_files, video_metric_directory)
test_data = load_data_from_files(test_files, video_metric_directory)

C:\Users\cpfit\AppData\Local\Temp\ipykernel_5496\4119103708.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(data_frames, ignore_index=True)
C:\Users\cpfit\AppData\Local\Temp\ipykernel_5496\4119103708.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(data_frames, ignore_index=True)


In [5]:
train_data.drop(columns=['Video_Path'], inplace=True)
test_data.drop(columns=['Video_Path'], inplace=True)

train_data = train_data.fillna(train_data.median())
test_data = test_data.fillna(test_data.median())

C:\Users\cpfit\AppData\Local\Temp\ipykernel_5496\2728043661.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_data = train_data.fillna(train_data.median())
C:\Users\cpfit\AppData\Local\Temp\ipykernel_5496\2728043661.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_data = test_data.fillna(test_data.median())


In [6]:
X_train = train_data.drop(columns=['Label'])
y_train = train_data['Label']
X_test = test_data.drop(columns=['Label'])
y_test = test_data['Label']

# Neural Network

In [7]:
# Function to create the model, required for KerasClassifier
def create_model(layers=[100], activation='relu', optimizer='adam', dropout_rate=0.2):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i == 0:
            model.add(Dense(nodes, input_dim=X_train.shape[1], activation=activation))
        else:
            model.add(Dense(nodes, activation=activation))
        model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [8]:
# Wrap Keras model so it can be used by scikit-learn
model = KerasClassifier(model=create_model, epochs=100, batch_size=32)

In [10]:
# Define the parameters range for random search
param_dist = {
    'model__layers': [(10,), (50,), (10, 10), (50, 50)],  # Different layer configurations
    'model__activation': ['relu', 'tanh', 'sigmoid'],
    'model__optimizer': ['adam', 'sgd'],
    'batch_size': [10, 20, 50],
    'epochs': [10, 50, 100]
}
# Create a RandomizedSearchCV object
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=5, cv=2, verbose=2, random_state=42)

In [11]:
# Fit the model
random_search.fit(X_train, y_train)

Fitting 2 folds for each of 5 candidates, totalling 10 fits
Epoch 1/100


c:\Users\cpfit\OneDrive\Documents\GitHub\cv-lie-detection\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


624/624 ━━━━━━━━━━━━━━━━━━━━ 1s 560us/step - accuracy: 0.4983 - loss: 0.7519
Epoch 2/100
624/624 ━━━━━━━━━━━━━━━━━━━━ 0s 531us/step - accuracy: 0.5115 - loss: 0.7098
Epoch 3/100
624/624 ━━━━━━━━━━━━━━━━━━━━ 0s 558us/step - accuracy: 0.5001 - loss: 0.7023
Epoch 4/100
624/624 ━━━━━━━━━━━━━━━━━━━━ 0s 528us/step - accuracy: 0.5128 - loss: 0.6966
Epoch 5/100
624/624 ━━━━━━━━━━━━━━━━━━━━ 0s 538us/step - accuracy: 0.5172 - loss: 0.6927
Epoch 6/100
624/624 ━━━━━━━━━━━━━━━━━━━━ 0s 567us/step - accuracy: 0.5244 - loss: 0.6901
Epoch 7/100
624/624 ━━━━━━━━━━━━━━━━━━━━ 0s 560us/step - accuracy: 0.5352 - loss: 0.6879
Epoch 8/100
624/624 ━━━━━━━━━━━━━━━━━━━━ 0s 523us/step - accuracy: 0.5383 - loss: 0.6871
Epoch 9/100
624/624 ━━━━━━━━━━━━━━━━━━━━ 0s 539us/step - accuracy: 0.5450 - loss: 0.6850
Epoch 10/100
624/624 ━━━━━━━━━━━━━━━━━━━━ 0s 563us/step - accuracy: 0.5392 - loss: 0.6867
Epoch 11/100
624/624 ━━━━━━━━━━━━━━━━━━━━ 0s 539us/step - accuracy: 0.5450 - loss: 0.6836
Epoch 12/100
624/624 ━━━━━━━━━━

c:\Users\cpfit\OneDrive\Documents\GitHub\cv-lie-detection\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


624/624 ━━━━━━━━━━━━━━━━━━━━ 1s 546us/step - accuracy: 0.4944 - loss: 0.7483
Epoch 2/100
624/624 ━━━━━━━━━━━━━━━━━━━━ 0s 543us/step - accuracy: 0.5044 - loss: 0.7186
Epoch 3/100
624/624 ━━━━━━━━━━━━━━━━━━━━ 0s 506us/step - accuracy: 0.5093 - loss: 0.7042
Epoch 4/100
624/624 ━━━━━━━━━━━━━━━━━━━━ 0s 547us/step - accuracy: 0.5048 - loss: 0.6988
Epoch 5/100
624/624 ━━━━━━━━━━━━━━━━━━━━ 0s 528us/step - accuracy: 0.5007 - loss: 0.6960
Epoch 6/100
624/624 ━━━━━━━━━━━━━━━━━━━━ 0s 537us/step - accuracy: 0.5116 - loss: 0.6939
Epoch 7/100
624/624 ━━━━━━━━━━━━━━━━━━━━ 0s 522us/step - accuracy: 0.5026 - loss: 0.6942
Epoch 8/100
624/624 ━━━━━━━━━━━━━━━━━━━━ 0s 549us/step - accuracy: 0.5123 - loss: 0.6929
Epoch 9/100
624/624 ━━━━━━━━━━━━━━━━━━━━ 0s 517us/step - accuracy: 0.5129 - loss: 0.6929
Epoch 10/100
624/624 ━━━━━━━━━━━━━━━━━━━━ 0s 553us/step - accuracy: 0.5189 - loss: 0.6928
Epoch 11/100
624/624 ━━━━━━━━━━━━━━━━━━━━ 0s 521us/step - accuracy: 0.5090 - loss: 0.6929
Epoch 12/100
624/624 ━━━━━━━━━━

...

c:\Users\cpfit\OneDrive\Documents\GitHub\cv-lie-detection\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3117/3117 ━━━━━━━━━━━━━━━━━━━━ 3s 651us/step - accuracy: 0.5246 - loss: 0.7154
Epoch 2/10
3117/3117 ━━━━━━━━━━━━━━━━━━━━ 2s 633us/step - accuracy: 0.5737 - loss: 0.6781
Epoch 3/10
3117/3117 ━━━━━━━━━━━━━━━━━━━━ 2s 650us/step - accuracy: 0.5916 - loss: 0.6698
Epoch 4/10
3117/3117 ━━━━━━━━━━━━━━━━━━━━ 2s 769us/step - accuracy: 0.5906 - loss: 0.6688
Epoch 5/10
3117/3117 ━━━━━━━━━━━━━━━━━━━━ 2s 708us/step - accuracy: 0.5902 - loss: 0.6690
Epoch 6/10
3117/3117 ━━━━━━━━━━━━━━━━━━━━ 2s 623us/step - accuracy: 0.5921 - loss: 0.6703
Epoch 7/10
3117/3117 ━━━━━━━━━━━━━━━━━━━━ 2s 664us/step - accuracy: 0.5942 - loss: 0.6685
Epoch 8/10
3117/3117 ━━━━━━━━━━━━━━━━━━━━ 2s 638us/step - accuracy: 0.5866 - loss: 0.6688
Epoch 9/10
3117/3117 ━━━━━━━━━━━━━━━━━━━━ 2s 620us/step - accuracy: 0.5989 - loss: 0.6634
Epoch 10/10
3117/3117 ━━━━━━━━━━━━━━━━━━━━ 2s 634us/step - accuracy: 0.5933 - loss: 0.6673
3117/3117 ━━━━━━━━━━━━━━━━━━━━ 2s 524us/step
[CV] END batch_size=10, epochs=10, model__activation=tanh, model_

c:\Users\cpfit\OneDrive\Documents\GitHub\cv-lie-detection\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3117/3117 ━━━━━━━━━━━━━━━━━━━━ 3s 684us/step - accuracy: 0.5737 - loss: 0.6755
Epoch 2/100
3117/3117 ━━━━━━━━━━━━━━━━━━━━ 2s 650us/step - accuracy: 0.6506 - loss: 0.6151
Epoch 3/100
3117/3117 ━━━━━━━━━━━━━━━━━━━━ 2s 645us/step - accuracy: 0.6582 - loss: 0.6083
Epoch 4/100
3117/3117 ━━━━━━━━━━━━━━━━━━━━ 2s 657us/step - accuracy: 0.6579 - loss: 0.6060
Epoch 5/100
3117/3117 ━━━━━━━━━━━━━━━━━━━━ 2s 646us/step - accuracy: 0.6628 - loss: 0.6015
Epoch 6/100
3117/3117 ━━━━━━━━━━━━━━━━━━━━ 2s 710us/step - accuracy: 0.6601 - loss: 0.6021
Epoch 7/100
3117/3117 ━━━━━━━━━━━━━━━━━━━━ 2s 670us/step - accuracy: 0.6636 - loss: 0.5977
Epoch 8/100
3117/3117 ━━━━━━━━━━━━━━━━━━━━ 2s 650us/step - accuracy: 0.6689 - loss: 0.5951
Epoch 9/100
3117/3117 ━━━━━━━━━━━━━━━━━━━━ 2s 643us/step - accuracy: 0.6661 - loss: 0.5947
Epoch 10/100
3117/3117 ━━━━━━━━━━━━━━━━━━━━ 2s 688us/step - accuracy: 0.6665 - loss: 0.5943
Epoch 11/100
3117/3117 ━━━━━━━━━━━━━━━━━━━━ 3s 837us/step - accuracy: 0.6682 - loss: 0.5948
Epoch 12/

RandomizedSearchCV(cv=2,
                   estimator=KerasClassifier(batch_size=32, epochs=100, model=<function create_model at 0x00000255CAE8A0C0>),
                   n_iter=5,
                   param_distributions={'batch_size': [10, 20, 50],
                                        'epochs': [10, 50, 100],
                                        'model__activation': ['relu', 'tanh',
                                                              'sigmoid'],
                                        'model__layers': [(10,), (50,),
                                                          (10, 10), (50, 50)],
                                        'model__optimizer': ['adam', 'sgd']},
                   random_state=42, verbose=2)

In [12]:
# Print the best parameters and the best score
print("Best parameters:", random_search.best_params_)
print("Best score:", random_search.best_score_)

# Evaluate the best model on the test set
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Test set accuracy:", accuracy)
print("Classification Report:\n", class_report)

Best parameters: {'model__optimizer': 'adam', 'model__layers': (50,), 'model__activation': 'sigmoid', 'epochs': 100, 'batch_size': 20}
Best score: 0.661506625172447
969/969 ━━━━━━━━━━━━━━━━━━━━ 1s 509us/step
Test set accuracy: 0.5826751329306695
Classification Report:
               precision    recall  f1-score   support

           0       0.57      0.81      0.67     10063
           1       0.62      0.33      0.43      9308

    accuracy                           0.58     19371
   macro avg       0.60      0.57      0.55     19371
weighted avg       0.59      0.58      0.56     19371

